In [1]:
#!python /home/gtamkin/_AGB-dev/mpf-model-factories/MultiPathFusion/multi_path_fusion/notebooks/glenn/gt_mc_prep_sets.py

In [2]:
bandLen = 0  # aggregate number of indices
batch_size = 0
max_num_occurences = 0

In [3]:
from random import seed
from random import randint
def randint_sample(bandLen, batch_size):
    random_set = []
    #seed()
    # generate some integers
    for _ in range(10):
     random_set.append(str(randint(1, bandLen)))
#    print(random_set)    
    return random_set

In [34]:
import numpy as np

from datetime import datetime
import socket

start_time = datetime.now()

spectrum = "NEON"
#spectrum = "VI"
#spectrum = "ALL"

if (spectrum == "ALL"):

    # aggregate number of indices (426 + 21)
    bandLen = 447  # aggregate number of indices
    batch_size = 10
#    max_num_occurences = 50
#    max_num_occurences = 200
    max_num_occurences = 10

elif (spectrum == "NEON"):

    # Full indices (#426)
    bandLen = 426  # number indices in index file (large file)
    batch_size = 10
    max_num_occurences = 10
#    max_num_occurences = 100
#    max_num_occurences = 50

else:

    # Partial indices (#21)
    bandLen = 21  # number of bands in original hyperspectral file
    batch_size = 10
    max_num_occurences = 50
#    max_num_occurences = 100
    max_num_occurences = 10

MLBS_2018_band_baseline = []
for i in range(bandLen):
    MLBS_2018_band_baseline.append(str(i + 1))
bands = MLBS_2018_band_baseline

# Generate sets of random indices

In [35]:
bandOccurenceArr = np.zeros(bandLen).astype(int)
print(bandOccurenceArr)
bandAbsSumArr = np.zeros(bandLen)
bandMaxArr = np.zeros(bandLen)
bandMinArr = np.zeros(bandLen)
bandAvgArr = np.zeros(bandLen)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [36]:
#from random import sample
random_sets = []
num_samples = 0

max_num_samples = 250000
not_finished = True
bandFoundArr = np.zeros(bandLen).astype(int)
maxed_out_list = []
lastSavedBands = None
shrinkSet = False
dump = True

while not_finished == True:
#    random_set = sample(bands, batch_size)
    random_set = randint_sample(bandLen, batch_size)
    random_sets.append(random_set)

    for i in range(len(random_set)):
        # get band from band list
        bandNum = random_set[i]

        # increment Occurence value in the cell that contains the band number
        bandFoundArr[int(bandNum)-1] = bandFoundArr[int(bandNum)-1] + 1
    
        for index in range(len(bandFoundArr)):
                if (bandFoundArr[index-1] > max_num_occurences):
                    if index not in maxed_out_list:
                        maxed_out_list.append(index)
                    if (shrinkSet == True):
                        print('remove: ', index)
                        bands.remove(index)
                        print('maxed_out_list of bands = ', maxed_out_list)

    num_samples = num_samples+1
    if (num_samples > max_num_samples):
        print(num_samples,' > ', max_num_samples)
        not_finished = False
        dump = False
        break;    
    if (len(maxed_out_list) == bandLen):
        print('Processing complete! len(maxed_out_list):', len(maxed_out_list))
        not_finished = False
        break;

print('final tally: ', bandFoundArr)
print('num random_sets:', len(random_sets))

if (dump == True):
    date = '20240220'
#    version = date + '_shrink_' + str(shrinkSet)
    version = date

    import pickle
    random_set_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_' + version + '_' \
        + 'collection-' + str(spectrum) + '_min_occurrences-' + str(max_num_occurences) + '_batch_size-' + str(batch_size) + '_num_samples-' + str(num_samples) + '_bandLen-' + str(bandLen) + '.pkl'
    # random_set_file = '/explore/nobackup/projects/ilab/data/AGB/test/mlruns/random_sets/random_sets_' + version + '_' \
    #     + 'collection_' + str(spectrum) + '_batch_size=' + str(batch_size) + '_num_samples=' + str(num_samples) + '_bandLen=' + str(bandLen) + '_' + version + '.pkl'
    pickle.dump(random_sets, open(random_set_file , "wb"))
    print('Saved {', str(len(random_sets)), '} random sets of {', batch_size,'} to file: ', random_set_file)

time_elapsed = datetime.now() - start_time
print('Time elapsed on node [{}] = (hh:mm:ss.ms) {}'.format(socket.gethostname(), time_elapsed))

print('\nrandom_sets[0]:', random_sets[0])


Processing complete! len(maxed_out_list): 426
final tally:  [31 33 28 28 23 27 26 23 25 23 16 33 24 23 24 27 17 23 24 20 18 25 32 19
 35 35 32 31 27 35 17 18 23 31 31 28 23 31 26 32 32 23 23 32 26 34 19 25
 27 24 17 18 27 26 26 23 25 30 28 27 32 25 30 20 25 24 35 22 33 24 36 20
 24 33 27 27 22 24 35 29 21 25 28 20 17 31 27 27 25 39 28 24 34 21 19 23
 18 28 21 16 27 21 29 18 22 33 21 20 23 25 21 36 28 25 21 26 25 32 34 24
 25 23 25 25 33 24 29 24 19 22 27 22 25 23 25 32 19 30 26 25 24 27 11 30
 24 31 28 20 17 29 17 27 16 22 24 20 23 21 20 20 25 24 22 23 38 27 34 27
 19 13 30 27 24 33 26 24 26 28 15 22 26 23 28 25 27 25 19 15 28 27 28 27
 23 24 22 30 30 37 28 21 36 27 26 35 23 31 26 25 21 26 28 20 26 20 26 28
 29 27 26 26 28 19 17 18 34 30 22 31 21 30 30 21 29 28 28 32 28 24 17 31
 35 27 27 17 17 21 28 25 35 30 23 16 22 23 30 28 38 35 25 21 27 23 18 31
 22 25 22 22 27 18 27 29 26 25 33 26 18 29 24 24 17 37 27 19 20 43 26 26
 29 30 20 20 28 30 28 22 30 21 23 25 29 30 24 23 20 28 16 23 28 

In [37]:
random_sets_r = pickle.load(open(random_set_file, "rb"))   
print(len(random_sets_r), random_sets_r[0])

1080 ['354', '87', '77', '198', '307', '112', '104', '320', '398', '406']


In [8]:
_random_set_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240217_collection-ALL_max_occurrences-200_batch_size-10_num_samples-10897_bandLen-447.pkl'
random_sets_r = pickle.load(open(_random_set_file, "rb"))   
print(len(random_sets_r),random_sets_r[0])

10897 ['42', '408', '361', '230', '138', '398', '162', '398', '187', '417']


In [ ]:
import pandas as pd
df = pd.DataFrame(random_sets_r)
df